# Thu thập và phân tích dữ liệu từ một page Facebook
# Bước 1: Thu thập dữ liệu 

# Sử dụng Facebook Scraper để thu thập

In [ ]:
%pip install facebook_scraper pandas numpy
from facebook_scraper import get_posts
import pandas as pd
import numpy as np
import os
import json

In [ ]:
FANPAGE_LINK ="Steam"
FOLDER_PATH = "Data/"
COOKIE_PATH = "./mbasic.facebook.com_cookies.txt"

PAGES_NUMBER = 100 # Number of pages to crawl

post_list = []
for post in get_posts(FANPAGE_LINK,
                    options={"comments": True, "reactions": True, "allow_extra_requests": True},
                    extra_info=True, pages=PAGES_NUMBER, cookies=COOKIE_PATH):
    print(post)
    post_list.append(post)

# Chuyển dict này thành Dataframe

In [ ]:
# Initialize dataframe to scrape Facebook post
post_df_full = pd.DataFrame(columns=post_list[0].keys(), index=range(len(post_list)), data=post_list)

# To df
path=FOLDER_PATH + "RawData.csv"
post_df_full.to_csv(path, index=False)
print(path)

In [164]:
# Giờ ta sẽ sử dụng Dataframe đã tạo Valve.csv để phân tích
df = pd.read_csv('Data/RawData.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               248 non-null    int64 
 1   name             248 non-null    object
 2   id_post          248 non-null    object
 3   created_time     248 non-null    object
 4   updated_time     248 non-null    object
 5   message          248 non-null    object
 6   type             248 non-null    object
 7   picture          140 non-null    object
 8   source           32 non-null     object
 9   properties       32 non-null     object
 10  like             248 non-null    int64 
 11  love             248 non-null    int64 
 12  haha             248 non-null    int64 
 13  wow              248 non-null    int64 
 14  sad              248 non-null    int64 
 15  angry            248 non-null    int64 
 16  comments         248 non-null    object
 17  total_reactions  248 non-null    in

# Bước 2: Làm sạch và tiền xử lý dữ liệu

In [165]:
#Drop những column không cần dùng và những column có quá nhiều giá trị NaN
df = df.dropna(thresh=10, axis='columns')
df.columns



Index(['id', 'name', 'id_post', 'created_time', 'updated_time', 'message',
       'type', 'picture', 'source', 'properties', 'like', 'love', 'haha',
       'wow', 'sad', 'angry', 'comments', 'total_reactions', 'total_shares',
       'total_comments'],
      dtype='object')

In [166]:
#Những cột này không có giá trị sử dụng khi phân tích dữ liệu
df = df.drop(columns=['source','updated_time','properties','id'])

In [167]:
df.head()

,name,id_post,created_time,message,type,picture,like,love,haha,wow,sad,angry,comments,total_reactions,total_shares,total_comments
0,Biết thế éo đi làm,101084031778928_880705467395095,2023-09-29T14:01:41+0000,Ảo tưởng ngày đầu đi làm #BTEDL #AgencyCam,video,https://scontent.fhan3-3.fna.fbcdn.net/v/t15.5...,510,7,416,1,4,0,"[{'created_time': '2023-09-29T17:32:35+0000', ...",938,13,13
1,Biết thế éo đi làm,101084031778928_880409834091325,2023-09-29T13:00:01+0000,Biết thế bảo sếp sớm hơn 🤣\r\n\r\nTag ngay đứa...,photo,https://scontent.fhan4-2.fna.fbcdn.net/v/t39.3...,2037,1,514,0,0,0,"[{'created_time': '2023-09-29T13:35:39+0000', ...",2552,0,13
2,Biết thế éo đi làm,101084031778928_880604397405202,2023-09-29T11:01:11+0000,Những người hôm nay c.á.u g.ắ.t là những người...,status,NaN,1177,4,663,2,67,3,"[{'created_time': '2023-09-29T11:01:35+0000', ...",1916,34,141
3,Biết thế éo đi làm,101084031778928_880504614081847,2023-09-29T07:19:12+0000,Muon ve qua,status,NaN,4227,52,1828,3,596,0,"[{'created_time': '2023-09-29T07:20:08+0000', ...",6706,331,809
4,Biết thế éo đi làm,101084031778928_880444224087886,2023-09-29T05:04:12+0000,Từ chối yêu để tập trung sự nghiệp. Nhưng vừa ...,status,NaN,1293,3,1606,2,102,0,"[{'created_time': '2023-09-29T05:49:20+0000', ...",3006,31,135


In [168]:
#Tìm những data bị NaN và xử lý 
df.isna().sum()

name                 0
id_post              0
created_time         0
message              0
type                 0
picture            108
like                 0
love                 0
haha                 0
wow                  0
sad                  0
angry                0
comments             0
total_reactions      0
total_shares         0
total_comments       0
dtype: int64

In [169]:
#Có thể thấy ở cột picture những giá trị NaN có nghĩa là post không có ảnh
df['picture'] = df['picture'].fillna(" ")

In [170]:
#Loại bỏ những giá trị bị duplicate (nếu có)
df.drop_duplicates()

,name,id_post,created_time,message,type,picture,like,love,haha,wow,sad,angry,comments,total_reactions,total_shares,total_comments
0,Biết thế éo đi làm,101084031778928_880705467395095,2023-09-29T14:01:41+0000,Ảo tưởng ngày đầu đi làm #BTEDL #AgencyCam,video,https://scontent.fhan3-3.fna.fbcdn.net/v/t15.5...,510,7,416,1,4,0,"[{'created_time': '2023-09-29T17:32:35+0000', ...",938,13,13
1,Biết thế éo đi làm,101084031778928_880409834091325,2023-09-29T13:00:01+0000,Biết thế bảo sếp sớm hơn 🤣\r\n\r\nTag ngay đứa...,photo,https://scontent.fhan4-2.fna.fbcdn.net/v/t39.3...,2037,1,514,0,0,0,"[{'created_time': '2023-09-29T13:35:39+0000', ...",2552,0,13
2,Biết thế éo đi làm,101084031778928_880604397405202,2023-09-29T11:01:11+0000,Những người hôm nay c.á.u g.ắ.t là những người...,status,,1177,4,663,2,67,3,"[{'created_time': '2023-09-29T11:01:35+0000', ...",1916,34,141
3,Biết thế éo đi làm,101084031778928_880504614081847,2023-09-29T07:19:12+0000,Muon ve qua,status,,4227,52,1828,3,596,0,"[{'created_time': '2023-09-29T07:20:08+0000', ...",6706,331,809
4,Biết thế éo đi làm,101084031778928_880444224087886,2023-09-29T05:04:12+0000,Từ chối yêu để tập trung sự nghiệp. Nhưng vừa ...,status,,1293,3,1606,2,102,0,"[{'created_time': '2023-09-29T05:49:20+0000', ...",3006,31,135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,Biết thế éo đi làm,101084031778928_853270790138563,2023-08-22T03:02:15+0000,"""Gặp nhau rồi thật khó l.ìa xa... bao tâm sự c...",photo,https://scontent.fhan3-3.fna.fbcdn.net/v/t39.3...,2699,19,3826,2,59,0,"[{'created_time': '2023-08-22T03:04:06+0000', ...",6605,130,465
244,Biết thế éo đi làm,101084031778928_853231410142501,2023-08-22T01:31:34+0000,Lương 5tr thì ăn bao nhiêu đậu đỏ cũng không c...,status,,2952,13,3009,4,526,1,"[{'created_time': '2023-08-22T01:32:12+0000', ...",6505,282,999
245,Biết thế éo đi làm,101084031778928_852840903514885,2023-08-21T13:05:19+0000,Người zô tri đi phỏng vấn kiểu: \r\n#BTEDL #ag...,video,https://scontent.fhan4-1.fna.fbcdn.net/v/t15.5...,1201,9,824,0,10,0,"[{'created_time': '2023-09-18T08:03:33+0000', ...",2044,15,12
246,Biết thế éo đi làm,101084031778928_852792610186381,2023-08-21T11:30:30+0000,Tôi sẽ chỉ lấy chồng sau khi đã tìm được công ...,photo,https://scontent.fhan3-4.fna.fbcdn.net/v/t39.3...,3575,77,4716,6,250,1,"[{'created_time': '2023-08-21T18:37:49+0000', ...",8625,751,1083


In [171]:
df['like'][0]

510

In [172]:
df.head(5)


,name,id_post,created_time,message,type,picture,like,love,haha,wow,sad,angry,comments,total_reactions,total_shares,total_comments
0,Biết thế éo đi làm,101084031778928_880705467395095,2023-09-29T14:01:41+0000,Ảo tưởng ngày đầu đi làm #BTEDL #AgencyCam,video,https://scontent.fhan3-3.fna.fbcdn.net/v/t15.5...,510,7,416,1,4,0,"[{'created_time': '2023-09-29T17:32:35+0000', ...",938,13,13
1,Biết thế éo đi làm,101084031778928_880409834091325,2023-09-29T13:00:01+0000,Biết thế bảo sếp sớm hơn 🤣\r\n\r\nTag ngay đứa...,photo,https://scontent.fhan4-2.fna.fbcdn.net/v/t39.3...,2037,1,514,0,0,0,"[{'created_time': '2023-09-29T13:35:39+0000', ...",2552,0,13
2,Biết thế éo đi làm,101084031778928_880604397405202,2023-09-29T11:01:11+0000,Những người hôm nay c.á.u g.ắ.t là những người...,status,,1177,4,663,2,67,3,"[{'created_time': '2023-09-29T11:01:35+0000', ...",1916,34,141
3,Biết thế éo đi làm,101084031778928_880504614081847,2023-09-29T07:19:12+0000,Muon ve qua,status,,4227,52,1828,3,596,0,"[{'created_time': '2023-09-29T07:20:08+0000', ...",6706,331,809
4,Biết thế éo đi làm,101084031778928_880444224087886,2023-09-29T05:04:12+0000,Từ chối yêu để tập trung sự nghiệp. Nhưng vừa ...,status,,1293,3,1606,2,102,0,"[{'created_time': '2023-09-29T05:49:20+0000', ...",3006,31,135


In [173]:
#Chuyển Data đã xử lý vào một file csv khác
df.to_csv('Data/Data_Final.csv', index=False)